## 폐암 수술 환자의 생존율 예측

In [1]:
import os

import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

# 필요한 라이브러리 불러옴
import numpy as np

device = torch.device("cuda:0" if torch.cuda.is_available() else 'cpu')
print("PyTorch version :", torch.__version__)
print("Is GPU :", device)

import pandas as pd

# 실행할 때마다 같은 결과를 출력하기 위해 설정하는 부분
np.random.seed(3)
torch.manual_seed(3)

'''
# 준비된 수술 환자 데이터를 불러옴
Data_set = np.loadtxt("../dataset/ThoraricSurgery.csv", delimiter=',')

# 환자의 기록과 수술 결과를 X와 Y로 구분하여 저장
X = Data_set[:, 0:17]
Y = Data_set[:, 17]
'''

# 준비된 수술 환자 데이터를 불러옴 by using pandas
#df = pd.read_csv("../dataset/ThoraricSurgery.csv",
#                      names=["a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "o", "p", "q", "class"])
df = pd.read_csv("../dataset/ThoraricSurgery.csv", header=None)
print(df.head())
df.rename(columns={17:"class"}, inplace=True)
print(df.head())

# 환자의 기록과 수술 결과를 X와 Y로 구분하여 저장
X = df.drop(['class'], axis=1, inplace=False).values
Y = df['class'].values

PyTorch version : 1.6.0
Is GPU : cuda
    0   1     2     3   4   5   6   7   8   9   10  11  12  13  14  15  16  17
0  293   1  3.80  2.80   0   0   0   0   0   0  12   0   0   0   1   0  62   0
1    1   2  2.88  2.16   1   0   0   0   1   1  14   0   0   0   1   0  60   0
2    8   2  3.19  2.50   1   0   0   0   1   0  11   0   0   1   1   0  66   1
3   14   2  3.98  3.06   2   0   0   0   1   1  14   0   0   0   1   0  80   1
4   17   2  2.21  1.88   0   0   1   0   0   0  12   0   0   0   1   0  56   0
     0  1     2     3  4  5  6  7  8  9  10  11  12  13  14  15  16  class
0  293  1  3.80  2.80  0  0  0  0  0  0  12   0   0   0   1   0  62      0
1    1  2  2.88  2.16  1  0  0  0  1  1  14   0   0   0   1   0  60      0
2    8  2  3.19  2.50  1  0  0  0  1  0  11   0   0   1   1   0  66      1
3   14  2  3.98  3.06  2  0  0  0  1  1  14   0   0   0   1   0  80      1
4   17  2  2.21  1.88  0  0  1  0  0  0  12   0   0   0   1   0  56      0


In [2]:
class Model(nn.Module):
    """
    """
    def __init__(self):
        super(Model, self).__init__()
        
        self.input_size = 17
        
        self.layers = nn.Sequential(
            nn.Linear(in_features=self.input_size, out_features=30, bias=True),
            nn.ReLU(inplace=True),
            nn.Linear(in_features=30, out_features=1, bias=True),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        out = self.layers(x)
        return out
        

In [3]:
model = Model()
model

Model(
  (layers): Sequential(
    (0): Linear(in_features=17, out_features=30, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=30, out_features=1, bias=True)
    (3): Sigmoid()
  )
)

In [4]:
epochs = 500
batch_size = 64

x = torch.from_numpy(X).type(torch.FloatTensor)
y = torch.from_numpy(Y).type(torch.FloatTensor)

dataset = TensorDataset(x, y)       
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)  
    
#criterion = nn.CrossEntropyLoss()
criterion = nn.BCELoss()
#criterion = nn.BCEWithLogitsLoss() # Sigmoid + BCELoss

#    optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), 
#                                 lr=config.learning_rate, betas=(0.9, 0.98), eps=1e-09)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, betas=(0.5, 0.999))
        
for epoch in range(epochs):
    phase = 'train'
    for k, [inputs, targets] in enumerate(train_loader):
        inputs = inputs.to(device)
        targets = targets.to(device)
        
        optimizer.zero_grad()
        
        with torch.set_grad_enabled(phase=='train'):
            output = model(inputs)            
            #targets = targets.type_as(output)
            
            loss = criterion(output, targets)
            #print(loss)            
            
            loss.backward(retain_graph=True)
            optimizer.step()

C:\Anaconda3\envs\ml\lib\site-packages\torch\nn\modules\loss.py:529: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!

In [9]:
prediction = model(x)
#prediction = torch.sigmoid(prediction)
prediction.detach().numpy()

array([[4.11161418e-05],
       [5.60352253e-03],
       [3.47300502e-03],
       [1.31494121e-03],
       [7.35357730e-03],
       [5.04033966e-03],
       [1.54803135e-03],
       [6.76929252e-03],
       [1.07829394e-02],
       [3.49811045e-03],
       [2.30100984e-03],
       [8.01397895e-04],
       [3.29172891e-03],
       [8.95467238e-04],
       [1.37781361e-04],
       [1.72388705e-03],
       [1.06208026e-03],
       [7.65564386e-04],
       [3.65102751e-04],
       [6.53968251e-04],
       [5.64250804e-04],
       [5.09931648e-04],
       [3.96225107e-04],
       [6.19504368e-04],
       [3.65528336e-04],
       [2.90773954e-04],
       [3.29710165e-04],
       [1.18474847e-04],
       [1.82845426e-04],
       [3.95731731e-05],
       [9.22664185e-05],
       [3.81999962e-05],
       [3.90452551e-05],
       [3.61677921e-05],
       [2.11145561e-05],
       [9.24270239e-07],
       [1.35663722e-05],
       [8.30451609e-06],
       [2.07549037e-05],
       [6.99239608e-05],
